电商18 180412126 王佳琦

本次作业参考：https://www.bbsmax.com/A/n2d9MB7gdD/ 和https://blog.csdn.net/winycg/article/details/90318371

## 1.导入手写数据集

In [2]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms

In [2]:
parser = argparse.ArgumentParser(description='PyTorch MNIST Example') #创建一个 ArgumentParser 对象。ArgumentParser 对象包含将命令行解析成 Python 数据类型所需的全部信息。
parser.add_argument('--batch-size', type=int, default=128, metavar='N',
                    help='input batch size for training (default: 128)')   #添加程序参数信息
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='enables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
args = parser.parse_args(args=[])  #这里进行了修改，如果不修改，会报错。
args.cuda = not args.no_cuda and torch.cuda.is_available()
print(args)   #参数
 
#Sets the seed for generating random numbers. And returns a torch._C.Generator object.
torch.manual_seed(args.seed)   #随机数
if args.cuda:
    torch.cuda.manual_seed(args.seed)    

Namespace(batch_size=128, cuda=False, epochs=10, log_interval=10, no_cuda=False, seed=1)


In [3]:
kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}
trainset = datasets.MNIST('/data', train=True, download=True,transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(
    trainset,
    batch_size=args.batch_size, shuffle=True, **kwargs)


In [8]:
testset= datasets.MNIST('/data', train=False, transform=transforms.ToTensor())
test_loader = torch.utils.data.DataLoader(
    testset,
    batch_size=args.batch_size, shuffle=True, **kwargs)
image, label = trainset[0]
print(len(trainset))
print(image.size())
image, label = testset[0]
print(len(testset))
print(image.size())

60000
torch.Size([1, 28, 28])
10000
torch.Size([1, 28, 28])


## 2. 定义VAE

In [9]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
 
        self.fc1 = nn.Linear(784, 400)         #全连接层
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)
 
        self.relu = nn.ReLU()                    #激活函数
        self.sigmoid = nn.Sigmoid()
 
    def encode(self, x):                          #编码
        h1 = self.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)
 
    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        eps = Variable(std.data.new(std.size()).normal_())
        return eps.mul(std).add_(mu)
 
    def decode(self, z):                        #解码
        h3 = self.relu(self.fc3(z))
        return self.sigmoid(self.fc4(h3))
 
    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar
 
model = VAE()
if args.cuda:
    model.cuda()

In [10]:
reconstruction_function = nn.BCELoss()
reconstruction_function.size_average = False
 
def loss_function(recon_x, x, mu, logvar):
    BCE = reconstruction_function(recon_x, x.view(-1, 784))
 
    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)
 
    return BCE + KLD
 
optimizer = optim.Adam(model.parameters(), lr=1e-3)

定义了损失函数。

In [24]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = Variable(data)
        if args.cuda:
            data = data.cuda()
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss #loss_sum += loss.data[0]

        #这是因为输出的loss的数据类型是Variable。而PyTorch的动态图机制就是通过Variable来构建图。
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss/ len(data)))
 
    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))
    fake_images = gen_imgs.view(-1, 1, 28, 28)
    save_image(fake_images, 'MNIST_FAKE/fake_images-{}.png'.format(epoch + 1))
 
def test(epoch):
    model.eval()
    test_loss = 0
    for data, _ in test_loader:
        if args.cuda:
            data = data.cuda()
        data = Variable(data, volatile=True)
        recon_batch, mu, logvar = model(data)
        test_loss += loss_function(recon_batch, data, mu, logvar)
 
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [25]:
for epoch in range(1, args.epochs + 1):
    train(epoch)
    test(epoch)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.002090
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.002095
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.002109
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.002062
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.002089
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.002034
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.002111
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.002040
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.002064
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.002029
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.002135
Train Epoch: 1 [14080/60000 (23%)]	Loss: 0.002087
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.002062
Train Epoch: 1 [16640/60000 (28%)]	Loss: 0.002107
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.002132
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.002060
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.002059
Train Epoch: 1 [21760/60000 (36%)]	Loss: 0.002189
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.002022
Train Epoch: 1 [24320/60000 (41%)]	Loss: 0.002023
Train Epoch: 1 [

<ipython-input-24-6fba016bd045>:31: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data = Variable(data, volatile=True)


====> Test set loss: 0.0021
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.002119
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.002061
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.002109
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.002052
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.002046
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.002032
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.002019
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.001993
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.002141
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.002094
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.002062
Train Epoch: 2 [14080/60000 (23%)]	Loss: 0.002117
Train Epoch: 2 [15360/60000 (26%)]	Loss: 0.002025
Train Epoch: 2 [16640/60000 (28%)]	Loss: 0.002057
Train Epoch: 2 [17920/60000 (30%)]	Loss: 0.002044
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.001980
Train Epoch: 2 [20480/60000 (34%)]	Loss: 0.002066
Train Epoch: 2 [21760/60000 (36%)]	Loss: 0.002073
Train Epoch: 2 [23040/60000 (38%)]	Loss: 0.002039
Train Epoch: 2 [24320/60000 (41%)]	Los

Train Epoch: 5 [25600/60000 (43%)]	Loss: 0.002103
Train Epoch: 5 [26880/60000 (45%)]	Loss: 0.002031
Train Epoch: 5 [28160/60000 (47%)]	Loss: 0.002049
Train Epoch: 5 [29440/60000 (49%)]	Loss: 0.002070
Train Epoch: 5 [30720/60000 (51%)]	Loss: 0.002014
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.002037
Train Epoch: 5 [33280/60000 (55%)]	Loss: 0.002061
Train Epoch: 5 [34560/60000 (58%)]	Loss: 0.002065
Train Epoch: 5 [35840/60000 (60%)]	Loss: 0.002049
Train Epoch: 5 [37120/60000 (62%)]	Loss: 0.002040
Train Epoch: 5 [38400/60000 (64%)]	Loss: 0.002068
Train Epoch: 5 [39680/60000 (66%)]	Loss: 0.002111
Train Epoch: 5 [40960/60000 (68%)]	Loss: 0.002045
Train Epoch: 5 [42240/60000 (70%)]	Loss: 0.002073
Train Epoch: 5 [43520/60000 (72%)]	Loss: 0.002130
Train Epoch: 5 [44800/60000 (75%)]	Loss: 0.002047
Train Epoch: 5 [46080/60000 (77%)]	Loss: 0.002007
Train Epoch: 5 [47360/60000 (79%)]	Loss: 0.002123
Train Epoch: 5 [48640/60000 (81%)]	Loss: 0.002069
Train Epoch: 5 [49920/60000 (83%)]	Loss: 0.002079


Train Epoch: 8 [51200/60000 (85%)]	Loss: 0.001973
Train Epoch: 8 [52480/60000 (87%)]	Loss: 0.001995
Train Epoch: 8 [53760/60000 (90%)]	Loss: 0.002081
Train Epoch: 8 [55040/60000 (92%)]	Loss: 0.002054
Train Epoch: 8 [56320/60000 (94%)]	Loss: 0.002143
Train Epoch: 8 [57600/60000 (96%)]	Loss: 0.002004
Train Epoch: 8 [58880/60000 (98%)]	Loss: 0.001975
====> Epoch: 8 Average loss: 0.0021
====> Test set loss: 0.0021
Train Epoch: 9 [0/60000 (0%)]	Loss: 0.002074
Train Epoch: 9 [1280/60000 (2%)]	Loss: 0.002049
Train Epoch: 9 [2560/60000 (4%)]	Loss: 0.002071
Train Epoch: 9 [3840/60000 (6%)]	Loss: 0.001993
Train Epoch: 9 [5120/60000 (9%)]	Loss: 0.002116
Train Epoch: 9 [6400/60000 (11%)]	Loss: 0.001995
Train Epoch: 9 [7680/60000 (13%)]	Loss: 0.002034
Train Epoch: 9 [8960/60000 (15%)]	Loss: 0.002004
Train Epoch: 9 [10240/60000 (17%)]	Loss: 0.002047
Train Epoch: 9 [11520/60000 (19%)]	Loss: 0.002073
Train Epoch: 9 [12800/60000 (21%)]	Loss: 0.002078
Train Epoch: 9 [14080/60000 (23%)]	Loss: 0.002065
Tr

## 3. 输出生成图像

In [3]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision.utils import save_image

In [4]:
device="cpu"

建立一个新的文件夹

In [38]:
sample_dir = 'samples'
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

定义参数

In [7]:
image_size = 784
h_dim = 400
z_dim = 20
num_epochs = 15
batch_size = 128
learning_rate = 1e-3

In [40]:
dataset = trainset
 
# 数据加载器
data_loader = torch.utils.data.DataLoader(dataset=dataset,
                                          batch_size=batch_size, 
                                          shuffle=True)


In [8]:
# VAE model
class VAE(nn.Module):
    def __init__(self, image_size=784, h_dim=400, z_dim=20):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(image_size, h_dim)
        self.fc2 = nn.Linear(h_dim, z_dim) # 均值 向量
        self.fc3 = nn.Linear(h_dim, z_dim) # 保准方差 向量
        self.fc4 = nn.Linear(z_dim, h_dim)
        self.fc5 = nn.Linear(h_dim, image_size)
        
    # 编码过程
    def encode(self, x):
        h = F.relu(self.fc1(x))
        return self.fc2(h), self.fc3(h)
    
    # 随机生成隐含向量
    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var/2)
        eps = torch.randn_like(std)
        return mu + eps * std
 
    # 解码过程
    def decode(self, z):
        h = F.relu(self.fc4(z))
        return F.sigmoid(self.fc5(h))
    
    # 整个前向传播过程：编码-》解码
    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_reconst = self.decode(z)
        return x_reconst, mu, log_var


In [44]:
# 实例化一个模型
model = VAE().to(device)
# 创建优化器
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for epoch in range(num_epochs):
    for i, (x, _) in enumerate(data_loader):
        # 获取样本，并前向传播
        x = x.to(device).view(-1, image_size)
        x_reconst, mu, log_var = model(x)
        
        # 计算重构损失和KL散度（KL散度用于衡量两种分布的相似程度）
        # KL散度的计算可以参考论文或者文章开头的链接
        reconst_loss = F.binary_cross_entropy(x_reconst, x, size_average=False)
        kl_div = - 0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
        
        # 反向传播和优化
        loss = reconst_loss + kl_div
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ("Epoch[{}/{}], Step [{}/{}], Reconst Loss: {:.4f}, KL Div: {:.4f}" 
                   .format(epoch+1, num_epochs, i+1, len(data_loader), reconst_loss.item(), kl_div.item()))
    
    # 利用训练的模型进行测试
    with torch.no_grad():
        # 随机生成的图像
        z = torch.randn(batch_size, z_dim).to(device)
        out = model.decode(z).view(-1, 1, 28, 28) # 第一维数据不变，后一维数据转化为(1, 28, 28)
        save_image(out, os.path.join(sample_dir, 'sampled-{}.png'.format(epoch+1)))
        # 重构的图像
        out, _, _ = model(x)
        x_concat = torch.cat([x.view(-1, 1, 28, 28), out.view(-1, 1, 28, 28)], dim=3)
        save_image(x_concat, os.path.join(sample_dir, 'reconst-{}.png'.format(epoch+1)))

Epoch[1/15], Step [100/469], Reconst Loss: 23024.3047, KL Div: 1279.8553
Epoch[1/15], Step [200/469], Reconst Loss: 17445.0176, KL Div: 2054.7214
Epoch[1/15], Step [300/469], Reconst Loss: 14989.7158, KL Div: 2397.6909
Epoch[1/15], Step [400/469], Reconst Loss: 14289.7920, KL Div: 2552.9219
Epoch[2/15], Step [100/469], Reconst Loss: 12673.1621, KL Div: 2745.3796
Epoch[2/15], Step [200/469], Reconst Loss: 12700.6367, KL Div: 2967.8745
Epoch[2/15], Step [300/469], Reconst Loss: 11851.6396, KL Div: 3136.5991
Epoch[2/15], Step [400/469], Reconst Loss: 12564.5088, KL Div: 3087.8352
Epoch[3/15], Step [100/469], Reconst Loss: 12041.7598, KL Div: 3142.9043
Epoch[3/15], Step [200/469], Reconst Loss: 11142.5586, KL Div: 3031.6726
Epoch[3/15], Step [300/469], Reconst Loss: 11204.2998, KL Div: 3187.9622
Epoch[3/15], Step [400/469], Reconst Loss: 11661.8887, KL Div: 3112.5471
Epoch[4/15], Step [100/469], Reconst Loss: 11244.3086, KL Div: 3212.1965
Epoch[4/15], Step [200/469], Reconst Loss: 11115.06

## 4. 参考上列代码做法处理cifar-10

In [5]:
import torch
import torchvision
import torchvision.transforms as transforms

In [6]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])#数据集加载时，默认的图片格式是 numpy，所以通过 transforms 转换成 Tensor。
                                                              #然后，再对输入图片进行标准化。

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [7]:
sample_dir = 'samples-cifar'
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [8]:
dataset = trainset
 
# 数据加载器
data_loader = torch.utils.data.DataLoader(dataset=dataset,
                                          batch_size=batch_size, 
                                          shuffle=True)

In [9]:
# VAE model
class VAE(nn.Module):
    def __init__(self, image_size=12288, h_dim=400, z_dim=20):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(image_size, h_dim)
        self.fc2 = nn.Linear(h_dim, z_dim) # 均值 向量
        self.fc3 = nn.Linear(h_dim, z_dim) # 保准方差 向量
        self.fc4 = nn.Linear(z_dim, h_dim)
        self.fc5 = nn.Linear(h_dim, image_size)
        
    # 编码过程
    def encode(self, x):
        h = F.relu(self.fc1(x))
        return self.fc2(h), self.fc3(h)
    
    # 随机生成隐含向量
    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var/2)
        eps = torch.randn_like(std)
        return mu + eps * std
 
    # 解码过程
    def decode(self, z):
        h = F.relu(self.fc4(z))
        return F.sigmoid(self.fc5(h))
    
    # 整个前向传播过程：编码-》解码
    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_reconst = self.decode(z)
        return x_reconst, mu, log_var

In [10]:
image_size = 12288
h_dim = 400
z_dim = 20
num_epochs = 15
batch_size = 128
learning_rate = 1e-3

In [11]:
device="cpu"
model = VAE().to(device)
# 创建优化器
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [12]:
for epoch in range(num_epochs):
    print("goingon")
    for i, (x, _) in enumerate(data_loader):
        # 获取样本，并前向传播
        x = x.to(device).view(-1, image_size)
        x_reconst, mu, log_var = model(x)
        
        # 计算重构损失和KL散度（KL散度用于衡量两种分布的相似程度）
        # KL散度的计算可以参考论文或者文章开头的链接
        reconst_loss = F.binary_cross_entropy(x_reconst, x, size_average=False)
        kl_div = - 0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
        
        # 反向传播和优化
        loss = reconst_loss + kl_div
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ("Epoch[{}/{}], Step [{}/{}], Reconst Loss: {:.4f}, KL Div: {:.4f}" 
                   .format(epoch+1, num_epochs, i+1, len(data_loader), reconst_loss.item(), kl_div.item()))
    
    # 利用训练的模型进行测试
    with torch.no_grad():
        # 随机生成的图像
        z = torch.randn(batch_size, z_dim).to(device)
        out = model.decode(z).view(-1, 1, 96, 128)
        save_image(out, os.path.join(sample_dir, 'sampled-{}.png'.format(epoch+1)))
        # 重构的图像
        out, _, _ = model(x)
        x_concat = torch.cat([x.view(-1, 1, 96, 128), out.view(-1, 1,96, 128)], dim=3)
        save_image(x_concat, os.path.join(sample_dir, 'reconst-{}.png'.format(epoch+1)))

goingon


D:\anaconda\envs\pytorch\lib\site-packages\torch\nn\functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
D:\anaconda\envs\pytorch\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Epoch[1/15], Step [100/12500], Reconst Loss: -52152.1406, KL Div: 1473.6368
Epoch[1/15], Step [200/12500], Reconst Loss: -190529.2969, KL Div: 4438.0088
Epoch[1/15], Step [300/12500], Reconst Loss: -42189.6016, KL Div: 1713.4611
Epoch[1/15], Step [400/12500], Reconst Loss: 6117.5293, KL Div: 1089.2856
Epoch[1/15], Step [500/12500], Reconst Loss: 218.0721, KL Div: 1142.3660
Epoch[1/15], Step [600/12500], Reconst Loss: -21092.1738, KL Div: 1548.6235
Epoch[1/15], Step [700/12500], Reconst Loss: -322608.9375, KL Div: 2766.1714
Epoch[1/15], Step [800/12500], Reconst Loss: 389.6350, KL Div: 378.6440
Epoch[1/15], Step [900/12500], Reconst Loss: -147971.6250, KL Div: 1097.3308
Epoch[1/15], Step [1000/12500], Reconst Loss: -7457.0908, KL Div: 481.3492
Epoch[1/15], Step [1100/12500], Reconst Loss: -107349.7188, KL Div: 875.4446
Epoch[1/15], Step [1200/12500], Reconst Loss: 4632.7217, KL Div: 1872.7836
Epoch[1/15], Step [1300/12500], Reconst Loss: -41779.0742, KL Div: 1027.6180
Epoch[1/15], Step 

Epoch[1/15], Step [10800/12500], Reconst Loss: -91991.7812, KL Div: 3575.6445
Epoch[1/15], Step [10900/12500], Reconst Loss: -53013.1523, KL Div: 1480.0315
Epoch[1/15], Step [11000/12500], Reconst Loss: -106524.1797, KL Div: 4558.2178
Epoch[1/15], Step [11100/12500], Reconst Loss: -229987.8594, KL Div: 4747.2598
Epoch[1/15], Step [11200/12500], Reconst Loss: -153046.3750, KL Div: 11504.8984
Epoch[1/15], Step [11300/12500], Reconst Loss: -29522.7441, KL Div: 6507.3315
Epoch[1/15], Step [11400/12500], Reconst Loss: -142111.2500, KL Div: 3627.5979
Epoch[1/15], Step [11500/12500], Reconst Loss: -6007.9639, KL Div: 9046.9023
Epoch[1/15], Step [11600/12500], Reconst Loss: -359269.0000, KL Div: 5103.2183
Epoch[1/15], Step [11700/12500], Reconst Loss: -27736.3496, KL Div: 11327.0400
Epoch[1/15], Step [11800/12500], Reconst Loss: -116286.6641, KL Div: 4542.6050
Epoch[1/15], Step [11900/12500], Reconst Loss: 7907.1978, KL Div: 7928.8257
Epoch[1/15], Step [12000/12500], Reconst Loss: -259339.7344

RuntimeError: all elements of input should be between 0 and 1